In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import visualization, preprocess, metrics, naive_autoencoder, analyze_compression, wip_model

2025-04-16 01:00:23.339431: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-16 01:00:23.381064: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-16 01:00:23.381086: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-16 01:00:23.381125: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-16 01:00:23.390370: I tensorflow/core/platform/cpu_feature_g

In [3]:
import tensorflow as tf
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("Built with GPU support:", tf.test.is_built_with_gpu_support())
print("GPU available:", tf.config.list_physical_devices('GPU'))

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18399609710395211606
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14211350528
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9328575417386626264
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4070 Ti SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]
Built with CUDA: True
Built with GPU support: True
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2025-04-16 01:00:27.554684: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-16 01:00:27.601410: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-16 01:00:27.601453: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-16 01:00:28.006417: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-16 01:00:28.006455: I tensorflow/compile

In [4]:
import gc
import tensorflow as tf

# Clear TF session
tf.keras.backend.clear_session()

# Run garbage collection
gc.collect()

20

In [5]:
sequence_length = 4
height = 64
width = 64
channels = 3

In [6]:
import tensorflow as tf

# Define your input shape for the video sequences
input_shape = (sequence_length, height, width, channels)

# Build encoder and decoder
encoder = wip_model.build_encoder(input_shape=input_shape, latent_channels=32)
decoder = wip_model.build_decoder(latent_shape=encoder.output_shape[1:], output_channels=3)

# Vector quantization layer
vq_layer = wip_model.VectorQuantizer(num_embeddings=256, embedding_dim=32)

# Get latent shape without embedding dimension for PixelCNNPrior input
z_index_shape = encoder.output_shape[1:-1]  # (T, H, W)
print("PixelCNN input shape (T, H, W):", z_index_shape)

# Build PixelCNN prior
pixelcnn_prior = wip_model.PixelCNNPrior(input_shape=z_index_shape, num_embeddings=256)

# Compose the full VQ-VAE module
vqvae = wip_model.VQVAEModule(
    encoder=encoder,
    decoder=decoder,
    vq_layer=vq_layer,
    pixelcnn_prior=pixelcnn_prior,
    beta=1.0
)

# Compile with optimizer
vqvae.compile(optimizer=tf.keras.optimizers.Adam(1e-4))

2025-04-16 01:00:35.743861: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-16 01:00:35.743938: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-16 01:00:35.743971: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-16 01:00:35.744125: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-16 01:00:35.744137: I tensorflow/core/co

PixelCNN input shape (T, H, W): (2, 16, 16)


In [8]:
import numpy as np

video_folder = "extracted_videos"

# Preprocess the videos
video_data, video_filenames = preprocess.preprocess_videos_with_mapping(
    video_folder=video_folder,
    sequence_length=sequence_length,
    height=height,
    width=width,
    channels=channels
)

# Split the data into training and validation sets
split_index = int(0.8 * len(video_data))
train_data = video_data[:split_index]
val_data = video_data[split_index:]

# Reshape the training and validation data to match the input shape of the autoencoder
train_data = train_data.reshape((-1, sequence_length, height, width, channels))
val_data = val_data.reshape((-1, sequence_length, height, width, channels))
train_data = train_data.astype(np.float32)
val_data = val_data.astype(np.float32)

print("Train data shape:", train_data.shape)  # Expected: (num_samples, sequence_length, height, width, channels)
print("Validation data shape:", val_data.shape)

# Preprocessing for WIP model, not necessary to run currently
import tensorflow as tf
import numpy as np

def make_dataset(array, batch_size=4, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices(array.astype(np.float32))
    if shuffle:
        ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = make_dataset(train_data, batch_size=8)
val_ds = make_dataset(val_data, batch_size=8, shuffle=False)
train_ds = tf.data.Dataset.from_tensor_slices(train_data).batch(8).prefetch(tf.data.AUTOTUNE)
val_ds   = tf.data.Dataset.from_tensor_slices(val_data).batch(8).prefetch(tf.data.AUTOTUNE)

Train data shape: (3128, 4, 64, 64, 3)
Validation data shape: (782, 4, 64, 64, 3)


In [ ]:
import os
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

# Output directory setup
run_name = "vqvae_video_compression"
output_dir = os.path.join("training_runs", run_name)
os.makedirs(output_dir, exist_ok=True)

# Callbacks
callbacks = [
    EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True),
    ModelCheckpoint(
    filepath=os.path.join(output_dir, "best_weights"),
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=True
    ),
    TensorBoard(log_dir=os.path.join(output_dir, "logs"))
]

history = vqvae.fit(
    train_ds,
    validation_data=val_ds,
    epochs=7,
    callbacks=callbacks
)

In [9]:
import os

# Loads the wip model weights
weights_path = os.path.join("training_runs", "vqvae_video_compression", "best_weights")
vqvae.load_weights(weights_path)
print("✓ Weights loaded from:", weights_path)

✓ Weights loaded from: training_runs/vqvae_video_compression/best_weights


In [10]:
metrics_val = metrics.evaluate_metrics_by_video(
    split="val",
    limit=3,
    video_data=video_data,
    video_filenames=video_filenames,
    autoencoder=vqvae,
    sequence_length=sequence_length,
    height=height,
    width=width
)
metrics_val

2025-04-16 01:03:47.886032: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2025-04-16 01:03:47.994097: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:231] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2025-04-16 01:03:47.994125: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:234] Used ptxas at ptxas
2025-04-16 01:03:47.994183: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-04-16 01:03:48.040911: W tensorflow/compiler/xla/stream_executor/gpu/redzone_allocator.cc:322] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2025-04-16 01:03:48.116223: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_k

27/99 [=======>......................] - ETA: 0s

2025-04-16 01:03:48.369726: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.


99/99 [==============================] - 1s 5ms/step
📹 Evaluating: celebv_-_Bdf9C0SAU_8.mp4 — 99 sequences
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/nchanak/miniconda3/envs/tf-gpu/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/nchanak/miniconda3/envs/tf-gpu/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/nchanak/miniconda3/envs/tf-gpu/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth


2025-04-16 01:03:49.599986: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-04-16 01:03:49.600169: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-04-16 01:03:49.600347: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-04-16 01:03:49.600391: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-04-16 01:03:49.601154: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-04-16

121/121 [==============================] - 1s 5ms/step
📹 Evaluating: celebv_-_Bdf9C0SAU_3.mp4 — 121 sequences
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/nchanak/miniconda3/envs/tf-gpu/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth
94/94 [==============================] - 1s 6ms/step
📹 Evaluating: celebv_-_Bdf9C0SAU_9.mp4 — 94 sequences
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/nchanak/miniconda3/envs/tf-gpu/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth


{'celebv_-_Bdf9C0SAU_8.mp4': {'psnr': 26.858744,
  'ssim': 0.6907409,
  'lpips': 0.2320663311114215},
 'celebv_-_Bdf9C0SAU_3.mp4': {'psnr': 27.104864,
  'ssim': 0.71721065,
  'lpips': 0.23042126326275267},
 'celebv_-_Bdf9C0SAU_9.mp4': {'psnr': 25.978458,
  'ssim': 0.7215844,
  'lpips': 0.22212070080035545}}

In [11]:
import importlib
import analyze_compression

importlib.reload(analyze_compression)
with tf.device('/CPU:0'):
    compression_results = analyze_compression.evaluate_compression_by_video(
        split="val",
        limit=3,
        video_data=video_data,
        video_filenames=video_filenames,
        autoencoder=vqvae,
        encoder=encoder,
        sequence_length=sequence_length,
        height=height,
        width=width
    )
compression_results

Evaluating compression on 3 val videos...

94/94 [==============================] - 2s 26ms/step


{'celebv_-_Bdf9C0SAU_8.mp4': {'video': 'celebv_-_Bdf9C0SAU_8.mp4',
  'num_sequences': 99,
  'num_frames': 396,
  'input_kb_estimated': 3282.0314071120415,
  'codec_kb': 36.412109375,
  'latent_kb_estimated': 13.292874336242676,
  'compression_ratio_codec': 90.13571208718368,
  'compression_ratio_latent': 246.90155974495812,
  'vq_time_secs': 3.201131820678711},
 'celebv_-_Bdf9C0SAU_3.mp4': {'video': 'celebv_-_Bdf9C0SAU_3.mp4',
  'num_sequences': 267,
  'num_frames': 1068,
  'input_kb_estimated': 9001.780255015248,
  'codec_kb': 134.998046875,
  'latent_kb_estimated': 34.84114074707031,
  'compression_ratio_codec': 66.68081845176879,
  'compression_ratio_latent': 258.3664042565019,
  'vq_time_secs': 7.992120742797852},
 'celebv_-_Bdf9C0SAU_9.mp4': {'video': 'celebv_-_Bdf9C0SAU_9.mp4',
  'num_sequences': 94,
  'num_frames': 376,
  'input_kb_estimated': 3259.8387960985624,
  'codec_kb': 31.8603515625,
  'latent_kb_estimated': 12.690022468566895,
  'compression_ratio_codec': 102.3164728645

In [ ]:
play_video = "celebv_-_Bdf9C0SAU_6.mp4"

visualization.generate_and_display_comparison(
    video_name=play_video,
    autoencoder=vqvae,
    sequence_length=sequence_length,
    height=height,
    width=width,
    cleanup=True 
)